In [9]:
import pandas as pd
import datetime

pd.set_option("display.max_columns",999)

import warnings
warnings.filterwarnings('ignore')

In [13]:
lease = pd.read_csv('lease_clean_data_sep24.csv', low_memory=False, 
                    keep_default_na = True).drop(columns='Unnamed: 0')

In [14]:
lease.head()

,leasedeal_id,property_id,renewal,sqft_min,sqft_max,date_on_market,date_off_market,days_on_market,actual_vacancy,from_date,to_date_,lease_sign_date,lease_expiration_date,lease_term_in_months,rate_actual,estimated_rent,service_type_id,property_type_id,location_occupancy_id,rba,tenant_improvement_allowance_per_sqft,free_months,construction_year,building_rating_id,research_market_name,cbsaid,cbsa_cities,cbsa_states,submarket_name,zip,currently_occupied_fl,year_on_market,year_off_market,delete_rec,new_from_date,new_to_date,year_from,year_to,years_occupied,actual_estimated_rent
0,110498312,157648,NaN,3650.0,3650.0,2006-03-31,2006-12-31,275.0,1.0,2006-10-05,2014-04-29,2006-09-05,2011-10-05,60.0,NaN,39.202936,4.0,5,111764312,3650.0,NaN,NaN,1928.0,3,New York City,35614.0,New York-Jersey City-White Plains,NY-NJ,Murray Hill,10016.0,N,2006.0,2006.0,N,2006-10-05,2014-04-29,2006,2014,8,NaN
1,30028220,76048,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1982-01-15,2001-04-15,1981-12-16,2000-04-14,219.0,NaN,NaN,11.0,5,30028220,NaN,NaN,NaN,1977.0,3,Kansas City,28140.0,Kansas City,MO-KS,Ward Parkway,64114.0,N,NaN,NaN,N,1982-01-15,2001-04-15,1982,2001,19,NaN
2,30187227,559441,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1986-01-15,2000-03-14,1985-12-16,2000-03-14,170.0,NaN,NaN,NaN,5,30187227,NaN,NaN,NaN,1985.0,2,Kansas City,28140.0,Kansas City,MO-KS,I-29 Corridor,64152.0,N,NaN,NaN,N,1986-01-15,2000-03-14,1986,2000,14,NaN
3,114096677,239837,NaN,1388.0,1388.0,2013-12-31,2014-03-31,90.0,1.0,2014-02-14,2015-02-12,2014-01-15,2015-02-13,12.0,NaN,18.914058,7.0,5,122849533,1388.0,NaN,NaN,1972.0,2,Houston,26420.0,Houston-The Woodlands-Sugar Land,TX,Riverway,77024.0,N,2013.0,2014.0,N,2014-02-14,2015-02-12,2014,2015,1,NaN
4,10587417,220914,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2001-06-30,NaN,2001-05-01,2006-06-30,60.0,NaN,21.278033,NaN,5,10587417,7200.0,NaN,NaN,1906.0,2,Boston,14460.0,Boston-Cambridge-Newton,MA-NH,Route 3 South,2382.0,Y,NaN,NaN,N,2001-06-30,2021-12-31,2001,2021,20,NaN


In [15]:
print(lease.shape)
lease.columns

(1067026, 40)


Index(['leasedeal_id', 'property_id', 'renewal', 'sqft_min', 'sqft_max',
       'date_on_market', 'date_off_market', 'days_on_market', 'actual_vacancy',
       'from_date', 'to_date_', 'lease_sign_date', 'lease_expiration_date',
       'lease_term_in_months', 'rate_actual', 'estimated_rent',
       'service_type_id', 'property_type_id', 'location_occupancy_id', 'rba',
       'tenant_improvement_allowance_per_sqft', 'free_months',
       'construction_year', 'building_rating_id', 'research_market_name',
       'cbsaid', 'cbsa_cities', 'cbsa_states', 'submarket_name', 'zip',
       'currently_occupied_fl', 'year_on_market', 'year_off_market',
       'delete_rec', 'new_from_date', 'new_to_date', 'year_from', 'year_to',
       'years_occupied', 'actual_estimated_rent'],
      dtype='object')

In [16]:
cnt_service= pd.DataFrame(lease.groupby(['cbsaid', 'submarket_name', 'property_type_id']).agg({'property_id': 'count'})).\
                          reset_index()
cnt_service = cnt_service.pivot(index=['cbsaid', 'submarket_name'], columns='property_type_id', values='property_id').reset_index()
cnt_service = cnt_service.add_prefix('property_type_id_').\
 rename(columns={'property_type_id_submarket_name':'submarket_name', 
                 'property_type_id_cbsaid':'cbsaid'})
cnt_service 

property_type_id,cbsaid,submarket_name,property_type_id_5
0,10100.0,Aberdeen,13
1,10140.0,Aberdeen,19
2,10180.0,Abilene,52
3,10180.0,Chimney Rock Area MF,2
4,10180.0,Downtown Abilene MF,41
...,...,...,...
3701,49700.0,Yuba County,7
3702,49740.0,Downtown Yuma MF,24
3703,49740.0,Yuma,18
3704,49780.0,Zanesville,47


In [18]:
cnt_building_rate= pd.DataFrame(lease.groupby(['cbsaid', 'submarket_name', 'building_rating_id']).\
                                agg({'property_id': 'count'})).\
                          reset_index()

cnt_building_rate = cnt_building_rate.pivot(index=['cbsaid', 'submarket_name'],
                                    columns='building_rating_id', values='property_id').reset_index()
cnt_building_rate = cnt_building_rate.add_prefix('building_rating_').\
        rename(columns={'building_rating_submarket_name':'submarket_name',
                       'building_rating_cbsaid':'cbsaid'})
cnt_building_rate 

building_rating_id,cbsaid,submarket_name,building_rating_1,building_rating_2,building_rating_3,building_rating_4,building_rating_5
0,10100.0,Aberdeen,1.0,9.0,3.0,NaN,NaN
1,10140.0,Aberdeen,NaN,10.0,9.0,NaN,NaN
2,10180.0,Abilene,NaN,40.0,12.0,NaN,NaN
3,10180.0,Chimney Rock Area MF,NaN,2.0,NaN,NaN,NaN
4,10180.0,Downtown Abilene MF,NaN,14.0,2.0,25.0,NaN
...,...,...,...,...,...,...,...
3701,49700.0,Yuba County,4.0,3.0,NaN,NaN,NaN
3702,49740.0,Downtown Yuma MF,1.0,17.0,6.0,NaN,NaN
3703,49740.0,Yuma,NaN,12.0,6.0,NaN,NaN
3704,49780.0,Zanesville,4.0,33.0,10.0,NaN,NaN


In [19]:
lease['const_2000_fl'] = lease.construction_year.apply(lambda x: 'Y' if x > 2000 else 'N')
cnt_const_year= pd.DataFrame(lease.groupby(['cbsaid', 'submarket_name', 'const_2000_fl']).\
                             agg({'property_id': 'count'})).\
                          reset_index()
cnt_const_year = cnt_const_year.pivot(index=['cbsaid', 'submarket_name'], columns='const_2000_fl', 
                                      values='property_id').reset_index()

cnt_const_year = cnt_const_year.add_prefix('construction_after_2000_').\
        rename(columns={'construction_after_2000_submarket_name':'submarket_name',
                       'construction_after_2000_cbsaid':'cbsaid'})
cnt_const_year

const_2000_fl,cbsaid,submarket_name,construction_after_2000_N,construction_after_2000_Y
0,10100.0,Aberdeen,11.0,2.0
1,10140.0,Aberdeen,17.0,2.0
2,10180.0,Abilene,45.0,7.0
3,10180.0,Chimney Rock Area MF,2.0,NaN
4,10180.0,Downtown Abilene MF,41.0,NaN
...,...,...,...,...
3701,49700.0,Yuba County,7.0,NaN
3702,49740.0,Downtown Yuma MF,13.0,11.0
3703,49740.0,Yuma,15.0,3.0
3704,49780.0,Zanesville,41.0,6.0


In [20]:
cnt_props = pd.DataFrame(lease.groupby(['cbsaid', 'submarket_name']).agg({'property_id': 'count'})).\
                          reset_index().rename(columns = {'property_id':'count_properties'})

cnt_props

,cbsaid,submarket_name,count_properties
0,10100.0,Aberdeen,13
1,10140.0,Aberdeen,19
2,10180.0,Abilene,52
3,10180.0,Chimney Rock Area MF,2
4,10180.0,Downtown Abilene MF,41
...,...,...,...
3701,49700.0,Yuba County,7
3702,49740.0,Downtown Yuma MF,24
3703,49740.0,Yuma,18
3704,49780.0,Zanesville,47


In [21]:
lease['renewal_fl'] = lease['renewal'].apply(lambda x:   'Y' if x is not None else 'N')
cnt_renewal= pd.DataFrame(lease.groupby(['cbsaid', 'submarket_name', 'renewal_fl']).agg({'property_id': 'count'})).\
                          reset_index()
cnt_renewal = cnt_renewal.pivot(index=['cbsaid','submarket_name'], columns='renewal_fl', values='property_id').reset_index()
cnt_renewal = cnt_renewal.add_prefix('renewal_').rename(columns={'renewal_submarket_name':'submarket_name',
                                                                'renewal_cbsaid':'cbsaid'})

cnt_renewal


renewal_fl,cbsaid,submarket_name,renewal_Y
0,10100.0,Aberdeen,13
1,10140.0,Aberdeen,19
2,10180.0,Abilene,52
3,10180.0,Chimney Rock Area MF,2
4,10180.0,Downtown Abilene MF,41
...,...,...,...
3701,49700.0,Yuba County,7
3702,49740.0,Downtown Yuma MF,24
3703,49740.0,Yuma,18
3704,49780.0,Zanesville,47


In [22]:
#remove outliers in vacant properties. Use for later
import plotly.express as px
vac= lease[lease['days_on_market'].notnull()]
vac.shape

(589103, 42)

In [23]:
vac['days_vacant'] = (pd.to_datetime(vac['date_off_market']) - pd.to_datetime(vac['date_on_market'])).dt.days
cnt_vacant_service= pd.DataFrame(vac.groupby(['cbsaid', 'submarket_name',  'property_type_id']).\
                                 agg({'days_vacant': 'mean'})).\
                          reset_index()
cnt_vacant_service = cnt_vacant_service.pivot(index=['cbsaid','submarket_name'], 
        columns='property_type_id', values='days_vacant').reset_index()
cnt_vacant_service = cnt_vacant_service.add_prefix('vacant_days_propertytype_').\
    rename(columns={'vacant_days_propertytype_submarket_name':'submarket_name',
                   'vacant_days_propertytype_cbsaid':'cbsaid'})

cnt_vacant_service 

property_type_id,cbsaid,submarket_name,vacant_days_propertytype_5
0,10100.0,Aberdeen,365.714286
1,10140.0,Aberdeen,662.000000
2,10180.0,Abilene,373.411765
3,10180.0,Chimney Rock Area MF,229.000000
4,10180.0,Downtown Abilene MF,524.483871
...,...,...,...
3542,49700.0,Yuba City,92.000000
3543,49700.0,Yuba County,300.285714
3544,49740.0,Downtown Yuma MF,298.086957
3545,49740.0,Yuma,281.166667


In [26]:
cnt_vacant_building_rating= pd.DataFrame(vac.groupby(['cbsaid', 'submarket_name', 'building_rating_id']).\
                                         agg({'days_vacant': 'mean'})).\
                          reset_index()
cnt_vacant_building_rating = cnt_vacant_building_rating.pivot(index=['cbsaid','submarket_name'], 
                                                              columns='building_rating_id', 
                                                              values='days_vacant').reset_index()
cnt_vacant_building_rating = cnt_vacant_building_rating.add_prefix('vacant_days_buildingrating_').\
        rename(columns={'vacant_days_buildingrating_submarket_name':'submarket_name',
                       'vacant_days_buildingrating_cbsaid':'cbsaid'})

cnt_vacant_building_rating


building_rating_id,cbsaid,submarket_name,vacant_days_buildingrating_1,vacant_days_buildingrating_2,vacant_days_buildingrating_3,vacant_days_buildingrating_4,vacant_days_buildingrating_5
0,10100.0,Aberdeen,914.00,297.000000,229.000000,NaN,NaN
1,10140.0,Aberdeen,NaN,502.750000,821.250000,NaN,NaN
2,10180.0,Abilene,NaN,416.000000,209.142857,NaN,NaN
3,10180.0,Chimney Rock Area MF,NaN,229.000000,NaN,NaN,NaN
4,10180.0,Downtown Abilene MF,NaN,502.250000,275.000000,553.166667,NaN
...,...,...,...,...,...,...,...
3542,49700.0,Yuba City,NaN,92.000000,NaN,NaN,NaN
3543,49700.0,Yuba County,365.75,213.000000,NaN,NaN,NaN
3544,49740.0,Downtown Yuma MF,549.00,311.823529,201.200000,NaN,NaN
3545,49740.0,Yuma,NaN,353.000000,137.500000,NaN,NaN


In [27]:
occupied = lease.copy()
occupied['new_to_date'] = occupied['new_to_date'].fillna('2021-09-01')
occupied[occupied['new_to_date'].isnull()]
occupied

,leasedeal_id,property_id,renewal,sqft_min,sqft_max,date_on_market,date_off_market,days_on_market,actual_vacancy,from_date,to_date_,lease_sign_date,lease_expiration_date,lease_term_in_months,rate_actual,estimated_rent,service_type_id,property_type_id,location_occupancy_id,rba,tenant_improvement_allowance_per_sqft,free_months,construction_year,building_rating_id,research_market_name,cbsaid,cbsa_cities,cbsa_states,submarket_name,zip,currently_occupied_fl,year_on_market,year_off_market,delete_rec,new_from_date,new_to_date,year_from,year_to,years_occupied,actual_estimated_rent,const_2000_fl,renewal_fl
0,110498312,157648,NaN,3650.0,3650.0,2006-03-31,2006-12-31,275.0,1.0,2006-10-05,2014-04-29,2006-09-05,2011-10-05,60.0,NaN,39.202936,4.0,5,111764312,3650.0,NaN,NaN,1928.0,3,New York City,35614.0,New York-Jersey City-White Plains,NY-NJ,Murray Hill,10016.0,N,2006.0,2006.0,N,2006-10-05,2014-04-29,2006,2014,8,NaN,N,Y
1,30028220,76048,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1982-01-15,2001-04-15,1981-12-16,2000-04-14,219.0,NaN,NaN,11.0,5,30028220,NaN,NaN,NaN,1977.0,3,Kansas City,28140.0,Kansas City,MO-KS,Ward Parkway,64114.0,N,NaN,NaN,N,1982-01-15,2001-04-15,1982,2001,19,NaN,N,Y
2,30187227,559441,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1986-01-15,2000-03-14,1985-12-16,2000-03-14,170.0,NaN,NaN,NaN,5,30187227,NaN,NaN,NaN,1985.0,2,Kansas City,28140.0,Kansas City,MO-KS,I-29 Corridor,64152.0,N,NaN,NaN,N,1986-01-15,2000-03-14,1986,2000,14,NaN,N,Y
3,114096677,239837,NaN,1388.0,1388.0,2013-12-31,2014-03-31,90.0,1.0,2014-02-14,2015-02-12,2014-01-15,2015-02-13,12.0,NaN,18.914058,7.0,5,122849533,1388.0,NaN,NaN,1972.0,2,Houston,26420.0,Houston-The Woodlands-Sugar Land,TX,Riverway,77024.0,N,2013.0,2014.0,N,2014-02-14,2015-02-12,2014,2015,1,NaN,N,Y
4,10587417,220914,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2001-06-30,NaN,2001-05-01,2006-06-30,60.0,NaN,21.278033,NaN,5,10587417,7200.0,NaN,NaN,1906.0,2,Boston,14460.0,Boston-Cambridge-Newton,MA-NH,Route 3 South,2382.0,Y,NaN,NaN,N,2001-06-30,2021-12-31,2001,2021,20,NaN,N,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1067021,111384471,300227,NaN,1750.0,1750.0,2008-06-30,2009-03-31,274.0,1.0,2009-01-01,2016-11-30,2008-11-18,2014-01-01,60.0,NaN,21.513206,4.0,5,114180680,1750.0,NaN,NaN,1965.0,3,Philadelphia,37980.0,Philadelphia-Camden-Wilmington,PA-NJ-DE-MD,Market Street West,19102.0,N,2008.0,2009.0,N,2009-01-01,2016-11-30,2009,2016,7,NaN,N,Y
1067022,30011451,61383,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1988-06-15,1988-06-15,1988-05-16,1997-12-14,114.0,NaN,NaN,11.0,5,30011451,NaN,NaN,NaN,1902.0,4,Pittsburgh,38300.0,Pittsburgh,PA,CBD,15219.0,N,NaN,NaN,N,1988-06-15,1988-06-15,1988,1988,0,NaN,N,Y
1067023,113201416,4157322,NaN,2500.0,2500.0,2011-06-30,2012-06-30,366.0,1.0,2012-05-15,NaN,2012-04-15,2015-05-14,36.0,15.0,15.071205,6.0,5,120804094,2500.0,NaN,NaN,1923.0,2,Chicago,16980.0,Chicago-Naperville-Elgin,IL-IN-WI,Fulton Market/Near West Side,60607.0,Y,2011.0,2012.0,N,2012-05-15,2021-12-31,2012,2021,9,0.995275,N,Y
1067024,10541376,303542,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2002-02-01,2002-03-04,2002-01-01,2003-01-31,12.0,NaN,32.398496,13.0,5,10541376,2395.0,NaN,NaN,1964.0,2,South Bay/San Jose,41940.0,San Jose-Sunnyvale-Santa Clara,CA,Los Gatos/Saratoga,95030.0,N,NaN,NaN,N,2002-02-01,2002-03-04,2002,2002,0,NaN,N,Y


In [28]:
occupied['days_occupied'] = (pd.to_datetime(occupied['new_to_date']) - pd.to_datetime(occupied['new_from_date'])).dt.days

cnt_occ_service= pd.DataFrame(occupied.groupby(['cbsaid','submarket_name', 'property_type_id']).\
                              agg({'days_occupied': 'mean'})).\
                          reset_index()
cnt_occ_service = cnt_occ_service.pivot(index=['cbsaid','submarket_name'], columns='property_type_id', 
                                        values='days_occupied').reset_index()
cnt_occ_service = cnt_occ_service.add_prefix('occupied_days_propertytype_').\
    rename(columns={'occupied_days_propertytype_submarket_name':'submarket_name',
                   'occupied_days_propertytype_cbsaid':'cbsaid'})

cnt_occ_service

property_type_id,cbsaid,submarket_name,occupied_days_propertytype_5
0,10100.0,Aberdeen,3297.230769
1,10140.0,Aberdeen,2561.052632
2,10180.0,Abilene,3641.288462
3,10180.0,Chimney Rock Area MF,4848.500000
4,10180.0,Downtown Abilene MF,2287.195122
...,...,...,...
3701,49700.0,Yuba County,3086.714286
3702,49740.0,Downtown Yuma MF,2388.666667
3703,49740.0,Yuma,2974.166667
3704,49780.0,Zanesville,2580.510638


In [29]:
cnt_occ_buildingrating = pd.DataFrame(occupied.groupby(['cbsaid','submarket_name', 'building_rating_id']).\
                                      agg({'days_occupied': 'mean'})).\
                          reset_index()
cnt_occ_buildingrating = cnt_occ_buildingrating.pivot(index=['cbsaid','submarket_name'], 
                                            columns='building_rating_id', values='days_occupied').reset_index()
cnt_occ_buildingrating = cnt_occ_buildingrating.add_prefix('occupied_days_buildingrating_').\
        rename(columns={'occupied_days_buildingrating_submarket_name':'submarket_name',
                       'occupied_days_buildingrating_cbsaid':'cbsaid'})

cnt_occ_buildingrating


building_rating_id,cbsaid,submarket_name,occupied_days_buildingrating_1,occupied_days_buildingrating_2,occupied_days_buildingrating_3,occupied_days_buildingrating_4,occupied_days_buildingrating_5
0,10100.0,Aberdeen,1238.00,3278.666667,4039.333333,NaN,NaN
1,10140.0,Aberdeen,NaN,2547.300000,2576.333333,NaN,NaN
2,10180.0,Abilene,NaN,4041.750000,2306.416667,NaN,NaN
3,10180.0,Chimney Rock Area MF,NaN,4848.500000,NaN,NaN,NaN
4,10180.0,Downtown Abilene MF,NaN,2870.714286,1263.500000,2042.32,NaN
...,...,...,...,...,...,...,...
3701,49700.0,Yuba County,3598.00,2405.000000,NaN,NaN,NaN
3702,49740.0,Downtown Yuma MF,3408.00,2762.470588,1159.666667,NaN,NaN
3703,49740.0,Yuma,NaN,3257.250000,2408.000000,NaN,NaN
3704,49780.0,Zanesville,3126.75,2488.121212,2666.900000,NaN,NaN


In [30]:
cnt_currently_occupied = pd.DataFrame(lease.groupby(['cbsaid','submarket_name', 'currently_occupied_fl']).\
                                      agg({'property_id': 'count'}).\
                          reset_index())
cnt_currently_occupied = cnt_currently_occupied.pivot(index=['cbsaid','submarket_name'], 
                                                      columns='currently_occupied_fl', values='property_id').reset_index()
cnt_currently_occupied = cnt_currently_occupied.add_prefix('currently_occupied_fl_').\
        rename(columns={'currently_occupied_fl_submarket_name':'submarket_name',
                       'currently_occupied_fl_cbsaid':'cbsaid'})

cnt_currently_occupied

currently_occupied_fl,cbsaid,submarket_name,currently_occupied_fl_N,currently_occupied_fl_Y
0,10100.0,Aberdeen,5.0,8.0
1,10140.0,Aberdeen,9.0,10.0
2,10180.0,Abilene,13.0,39.0
3,10180.0,Chimney Rock Area MF,NaN,2.0
4,10180.0,Downtown Abilene MF,10.0,31.0
...,...,...,...,...
3701,49700.0,Yuba County,1.0,6.0
3702,49740.0,Downtown Yuma MF,8.0,16.0
3703,49740.0,Yuma,2.0,16.0
3704,49780.0,Zanesville,10.0,37.0


In [31]:
cnt_avg_rent_building_rating = pd.DataFrame(lease.groupby(['cbsaid','submarket_name', 'building_rating_id']).\
                                      agg({'estimated_rent': 'mean'}).\
                          reset_index())
cnt_avg_rent_building_rating = cnt_avg_rent_building_rating.pivot(index=['cbsaid','submarket_name'], 
                                    columns='building_rating_id', values='estimated_rent').reset_index()
cnt_avg_rent_building_rating = cnt_avg_rent_building_rating.add_prefix('avg_rent_buildingrating_').\
        rename(columns={'avg_rent_buildingrating_submarket_name':'submarket_name',
                       'avg_rent_buildingrating_cbsaid':'cbsaid'})

cnt_avg_rent_building_rating


building_rating_id,cbsaid,submarket_name,avg_rent_buildingrating_1,avg_rent_buildingrating_2,avg_rent_buildingrating_3,avg_rent_buildingrating_4,avg_rent_buildingrating_5
0,10100.0,Aberdeen,12.425771,12.624866,12.970470,NaN,NaN
1,10140.0,Aberdeen,NaN,16.366925,17.534617,NaN,NaN
2,10180.0,Abilene,NaN,11.751387,14.080592,NaN,NaN
3,10180.0,Chimney Rock Area MF,NaN,12.777495,NaN,NaN,NaN
4,10180.0,Downtown Abilene MF,NaN,10.670866,12.764639,13.225918,NaN
...,...,...,...,...,...,...,...
3701,49700.0,Yuba County,11.876562,12.381419,NaN,NaN,NaN
3702,49740.0,Downtown Yuma MF,17.465653,16.742804,27.180419,NaN,NaN
3703,49740.0,Yuma,NaN,16.435734,20.225657,NaN,NaN
3704,49780.0,Zanesville,11.380579,13.444017,15.771025,NaN,NaN


In [32]:
cnt_avg_rent = pd.DataFrame(lease.groupby(['cbsaid','submarket_name']).\
                                      agg({'estimated_rent': 'mean'}).\
                          reset_index())
cnt_avg_rent = cnt_avg_rent.rename(columns = {'estimated_rent': 'avg_estimated_rent'}) 
cnt_avg_rent

,cbsaid,submarket_name,avg_estimated_rent
0,10100.0,Aberdeen,12.689305
1,10140.0,Aberdeen,16.920042
2,10180.0,Abilene,12.285163
3,10180.0,Chimney Rock Area MF,12.777495
4,10180.0,Downtown Abilene MF,12.260305
...,...,...,...
3701,49700.0,Yuba County,12.092929
3702,49740.0,Downtown Yuma MF,19.382326
3703,49740.0,Yuma,17.518569
3704,49780.0,Zanesville,13.770459


In [33]:
final = pd.merge(cnt_props, cnt_const_year, on = ['cbsaid','submarket_name'], how='left')
final = pd.merge(final, cnt_service, on = ['cbsaid','submarket_name'], how='left')
final = pd.merge(final, cnt_building_rate, on = ['cbsaid','submarket_name'], how='left')
final = pd.merge(final, cnt_renewal, on = ['cbsaid','submarket_name'], how='left')
final = pd.merge(final, cnt_vacant_service, on = ['cbsaid','submarket_name'], how='left')
final = pd.merge(final, cnt_vacant_building_rating, on = ['cbsaid','submarket_name'], how='left')
final = pd.merge(final, cnt_occ_service, on = ['cbsaid','submarket_name'], how='left')
final = pd.merge(final, cnt_occ_buildingrating, on = ['cbsaid','submarket_name'], how='left')
final = pd.merge(final, cnt_currently_occupied, on = ['cbsaid','submarket_name'], how='left')
final = pd.merge(final, cnt_avg_rent_building_rating, on = ['cbsaid','submarket_name'], how='left')
final = pd.merge(final, cnt_avg_rent, on = ['cbsaid','submarket_name'], how='left')

final

,cbsaid,submarket_name,count_properties,construction_after_2000_N,construction_after_2000_Y,property_type_id_5,building_rating_1,building_rating_2,building_rating_3,building_rating_4,building_rating_5,renewal_Y,vacant_days_propertytype_5,vacant_days_buildingrating_1,vacant_days_buildingrating_2,vacant_days_buildingrating_3,vacant_days_buildingrating_4,vacant_days_buildingrating_5,occupied_days_propertytype_5,occupied_days_buildingrating_1,occupied_days_buildingrating_2,occupied_days_buildingrating_3,occupied_days_buildingrating_4,occupied_days_buildingrating_5,currently_occupied_fl_N,currently_occupied_fl_Y,avg_rent_buildingrating_1,avg_rent_buildingrating_2,avg_rent_buildingrating_3,avg_rent_buildingrating_4,avg_rent_buildingrating_5,avg_estimated_rent
0,10100.0,Aberdeen,13,11.0,2.0,13,1.0,9.0,3.0,NaN,NaN,13,365.714286,914.00,297.000000,229.000000,NaN,NaN,3297.230769,1238.00,3278.666667,4039.333333,NaN,NaN,5.0,8.0,12.425771,12.624866,12.970470,NaN,NaN,12.689305
1,10140.0,Aberdeen,19,17.0,2.0,19,NaN,10.0,9.0,NaN,NaN,19,662.000000,NaN,502.750000,821.250000,NaN,NaN,2561.052632,NaN,2547.300000,2576.333333,NaN,NaN,9.0,10.0,NaN,16.366925,17.534617,NaN,NaN,16.920042
2,10180.0,Abilene,52,45.0,7.0,52,NaN,40.0,12.0,NaN,NaN,52,373.411765,NaN,416.000000,209.142857,NaN,NaN,3641.288462,NaN,4041.750000,2306.416667,NaN,NaN,13.0,39.0,NaN,11.751387,14.080592,NaN,NaN,12.285163
3,10180.0,Chimney Rock Area MF,2,2.0,NaN,2,NaN,2.0,NaN,NaN,NaN,2,229.000000,NaN,229.000000,NaN,NaN,NaN,4848.500000,NaN,4848.500000,NaN,NaN,NaN,NaN,2.0,NaN,12.777495,NaN,NaN,NaN,12.777495
4,10180.0,Downtown Abilene MF,41,41.0,NaN,41,NaN,14.0,2.0,25.0,NaN,41,524.483871,NaN,502.250000,275.000000,553.166667,NaN,2287.195122,NaN,2870.714286,1263.500000,2042.32,NaN,10.0,31.0,NaN,10.670866,12.764639,13.225918,NaN,12.260305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3701,49700.0,Yuba County,7,7.0,NaN,7,4.0,3.0,NaN,NaN,NaN,7,300.285714,365.75,213.000000,NaN,NaN,NaN,3086.714286,3598.00,2405.000000,NaN,NaN,NaN,1.0,6.0,11.876562,12.381419,NaN,NaN,NaN,12.092929
3702,49740.0,Downtown Yuma MF,24,13.0,11.0,24,1.0,17.0,6.0,NaN,NaN,24,298.086957,549.00,311.823529,201.200000,NaN,NaN,2388.666667,3408.00,2762.470588,1159.666667,NaN,NaN,8.0,16.0,17.465653,16.742804,27.180419,NaN,NaN,19.382326
3703,49740.0,Yuma,18,15.0,3.0,18,NaN,12.0,6.0,NaN,NaN,18,281.166667,NaN,353.000000,137.500000,NaN,NaN,2974.166667,NaN,3257.250000,2408.000000,NaN,NaN,2.0,16.0,NaN,16.435734,20.225657,NaN,NaN,17.518569
3704,49780.0,Zanesville,47,41.0,6.0,47,4.0,33.0,10.0,NaN,NaN,47,749.323529,730.25,771.148148,578.333333,NaN,NaN,2580.510638,3126.75,2488.121212,2666.900000,NaN,NaN,10.0,37.0,11.380579,13.444017,15.771025,NaN,NaN,13.770459


In [34]:
final.shape

(3706, 32)

In [35]:
final.columns

Index(['cbsaid', 'submarket_name', 'count_properties',
       'construction_after_2000_N', 'construction_after_2000_Y',
       'property_type_id_5', 'building_rating_1', 'building_rating_2',
       'building_rating_3', 'building_rating_4', 'building_rating_5',
       'renewal_Y', 'vacant_days_propertytype_5',
       'vacant_days_buildingrating_1', 'vacant_days_buildingrating_2',
       'vacant_days_buildingrating_3', 'vacant_days_buildingrating_4',
       'vacant_days_buildingrating_5', 'occupied_days_propertytype_5',
       'occupied_days_buildingrating_1', 'occupied_days_buildingrating_2',
       'occupied_days_buildingrating_3', 'occupied_days_buildingrating_4',
       'occupied_days_buildingrating_5', 'currently_occupied_fl_N',
       'currently_occupied_fl_Y', 'avg_rent_buildingrating_1',
       'avg_rent_buildingrating_2', 'avg_rent_buildingrating_3',
       'avg_rent_buildingrating_4', 'avg_rent_buildingrating_5',
       'avg_estimated_rent'],
      dtype='object')

In [36]:
#final.to_csv('./Costar_tables/cluster_submarket.csv')